In [1]:
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Open the file in read mode
with open('../corpus.txt', 'r') as file:
    # Read the lines from the file and store them in a list
    corpus = file.readlines()

# Optionally, you can remove newline characters from each line
corpus = [line.strip() for line in corpus]



query = "Somebody breached our contract and caused financial loss. What legal actions can we take?"

In [2]:
# Sample Legal document corpus for demonstration
legal_documents = corpus  # Assuming 'corpus' is defined elsewhere

# User's issue as input query
user_issue = query

# TF-IDF Vectorizer for document-query similarity
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.8, max_features=500, stop_words='english')
combined_corpus = legal_documents + [user_issue]
combined_tfidf_matrix = vectorizer.fit_transform(combined_corpus)

document_tfidf_matrix = combined_tfidf_matrix[:-1]
query_tfidf_matrix = combined_tfidf_matrix[-1]

similarity_scores = cosine_similarity(document_tfidf_matrix, query_tfidf_matrix).flatten()

# Flower Pollination Algorithm Parameters
num_flowers = 10
num_iterations = 20
top_k = 3
p = 0.8  # switch probability for global vs local pollination

# Initialize flowers (document combinations) based on top-ranked documents
initial_candidates = np.argsort(similarity_scores)[-top_k:]
flowers = [list(np.random.choice(initial_candidates, size=top_k, replace=False)) for _ in range(num_flowers)]

# Function to calculate fitness (similarity score sum)
def fitness(flower):
    return sum(similarity_scores[flower])

# Levy flight function for global pollination
def levy_flight(flower):
    new_flower = flower.copy()
    new_flower[random.randint(0, top_k - 1)] = random.choice(range(len(legal_documents)))
    return new_flower

# Flower Pollination Algorithm Optimization
for iteration in range(num_iterations):
    global_best = max(flowers, key=fitness)

    for i in range(num_flowers):
        if random.random() < p:  # global pollination
            new_flower = levy_flight(global_best)
        else:  # local pollination
            partner = flowers[random.randint(0, num_flowers - 1)]
            new_flower = flowers[i].copy()
            new_flower[random.randint(0, top_k - 1)] = partner[random.randint(0, top_k - 1)]

        # Update flower if the new position is better
        if fitness(new_flower) > fitness(flowers[i]):
            flowers[i] = new_flower

# Output the best-performing flower
best_flower = max(flowers, key=fitness)
best_documents = [legal_documents[i] for i in best_flower]
for i in best_documents:
    print(i, end='\n\n')

# Performance Metrics Calculation

# Define a relevance threshold to consider a document relevant
relevance_threshold = 0.15929  # Adjust based on similarity score distribution

# Rank documents by similarity score
ranked_indices = np.argsort(similarity_scores)[::-1]  # Indices sorted in descending order
ranked_scores = similarity_scores[ranked_indices]

# Determine relevance based on the adjusted threshold
relevance_labels = [1 if score >= relevance_threshold else 0 for score in ranked_scores]

# Calculate Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(relevance_labels):
    for rank, label in enumerate(relevance_labels, start=1):
        if label == 1:  # First relevant document
            return 1 / rank
    return 0  # No relevant document found

# Calculate Mean Average Precision (MAP)
def mean_average_precision(relevance_labels):
    relevant_docs = 0
    cumulative_precision = 0
    for rank, label in enumerate(relevance_labels, start=1):
        if label == 1:
            relevant_docs += 1
            cumulative_precision += relevant_docs / rank
    return cumulative_precision / relevant_docs if relevant_docs > 0 else 0

# Calculate MRR and MAP
mrr = mean_reciprocal_rank(relevance_labels)
map_score = mean_average_precision(relevance_labels)

print(f"\nMean Reciprocal Rank (MRR): {mrr:.2f}")
print(f"Mean Average Precision (MAP): {map_score:.2f}")


Section 101, Contract Law: Governs contract formation requirements, including offer, acceptance, and consideration. Subsection 101.1 details enforceable contract types. Subsection 101.2 addresses remedies for breach, specifying compensatory, punitive, and nominal damages.

Section 101, Contract Law: Governs contract formation requirements, including offer, acceptance, and consideration. Subsection 101.1 details enforceable contract types. Subsection 101.2 addresses remedies for breach, specifying compensatory, punitive, and nominal damages.

Section 101, Contract Law: Governs contract formation requirements, including offer, acceptance, and consideration. Subsection 101.1 details enforceable contract types. Subsection 101.2 addresses remedies for breach, specifying compensatory, punitive, and nominal damages.


Mean Reciprocal Rank (MRR): 0.00
Mean Average Precision (MAP): 0.00
